### Before continuing, make sure you have Wiki DB installed and in the same directory as this program. See README for link

## Description of Data Set and CNN goal
The Convolutional Neural Network (CNN) built in this project aims to recognize and classify images of faces based on the provided metadata. <br>
Specifically, the CNN will be trained to:
Identify the Gender and Predict the Age of a person based on their face.

## Import Dependencies 

In [ ]:
import scipy.io
import os
import matplotlib.pyplot as plt
from PIL import Image
from datetime import datetime, timedelta
import splitfolders
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from keras.models import Model




## Load Relevant Metadata

In [ ]:
# Path to the local dataset directory
import numpy as np


data_dir = './wiki'

# Path to the metadata file
metadata_file = os.path.join(data_dir, 'wiki.mat')

# Load the metadata from the .mat file
metadata = scipy.io.loadmat(metadata_file)
# Extract the relevant metadata
wiki_data = metadata['wiki'][0, 0]
all_image_paths = wiki_data['full_path'][0]
print(all_image_paths)
names = wiki_data['name'][0]
genders = wiki_data['gender'][0]
birthdates = wiki_data['dob'][0]
photo_taken = wiki_data['photo_taken'][0]

# Calculate age
# Function to convert MATLAB serial date number to a readable date
def convert_dob(dob):
    reference_date = datetime(1, 1, 1)
    days_since_reference = int(dob)
    actual_date = reference_date + timedelta(days=days_since_reference - 366)
    return actual_date.strftime("%Y-%m-%d")  # Format as Year-Month-Day

# Calculate ages
ages = []
for dob, photo_year in zip(birthdates, photo_taken):
    birthdate = convert_dob(dob)
    age = photo_year - int(birthdate[:4])

    ages.append(age)

ages = np.array(ages)
# MATLAB's reference date (0 days since year 0)
reference_date = datetime(1, 1, 1)



## Visualize 5 examples 

In [ ]:

# Function to display an image with its metadata
def display_image_with_metadata(image_path, name, gender, dob, photo_taken, age):
    # Construct the full image path
    full_image_path = os.path.join(data_dir, image_path[0])
    birthdate_str = convert_dob(dob)

    # Open and display the image
    image = Image.open(full_image_path)
    plt.imshow(image)
    plt.imshow(image)
    title = (f"Name: {name[0]}\n"
            f"Gender: {'Male' if gender == 1 else 'Female' }\n"
             f"Birthdate: {birthdate_str}\n"
             f"Year Picture was taken: {photo_taken}\n"
             # Can be off by 1 year, since db does not have month/day picture was taken
             f"Age when picture taken: {age}\n"
             )
    plt.title(title)
    plt.axis('off')
    # plt.show()

# Display a few images with their metadata
for i in range(5):
    display_image_with_metadata(all_image_paths[i], names[i], genders[i], birthdates[i], photo_taken[i], ages[i])
    

## Split Data


In [ ]:
# Split data between training and testing
# Check if the directory with split data already exists
if not os.path.exists("./wiki_split_data"):
    # Split data between training and testing
    splitfolders.ratio('./wiki', output="./wiki_split_data", seed=1337, ratio=(0.7, 0.15, 0.15))


## Build Model

In [ ]:
# Input layer
input_layer = Input(shape=(64, 64, 3))

# Shared layers
x = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(input_layer)
x = Flatten()(x)
x = Dense(units=128, activation='relu')(x)

# Output layers
age_output = Dense(units=1, activation='linear', name='age_output')(x)  # Regression for age
gender_output = Dense(units=1, activation='sigmoid', name='gender_output')(x)  # Binary classification for gender

# Define the model
model = Model(inputs=input_layer, outputs=[age_output, gender_output])

# Print the model summary
model.summary()

## Train Model

In [ ]:
import os
import numpy as np

def extract_labels(image_paths, all_image_paths, ages, genders):
    # Create empty lists to hold the extracted ages and genders
    extracted_ages = []
    extracted_genders = []

    # Loop through the image paths in the split dataset
    for path in image_paths:
        # Print the path being checked
        print(f'Checking path: {path}')
        # Get the index of the image path in the original metadata
        index = np.where(all_image_paths == path) # Adjusted to use path directly

        if len(index) > 0:
            print('Image found')
            extracted_ages.append(ages[index[0]])  # Append the age
            extracted_genders.append(genders[index[0]])  # Append the gender
        else:
            print(f'No match found for: {path}')  # Print if no match is found

    return np.array(extracted_ages), np.array(extracted_genders)

# Function to get all image paths in a directory
def get_image_paths(directory):
    image_paths = []
    for root, _, files in os.walk(directory):  # Walk through directory
        for file in files:
            print(f'Checking file: {file} in {root[-2:]}')
            if file.endswith(('.jpg')):  # Include only image files
                print(f'Found image: {file}')
                # add name of folder to image path
                image_paths.append(f"{root[-2:]}/{file}")  # Join path correctly
    return image_paths

# Use the function to get training, validation, and test image paths
train_image_paths = get_image_paths('./wiki_split_data/train')
val_image_paths = get_image_paths('./wiki_split_data/val')
test_image_paths = get_image_paths('./wiki_split_data/test')

# Print out the paths to verify
print("Train Image Paths:", train_image_paths)
print("Validation Image Paths:", val_image_paths)
print("Test Image Paths:", test_image_paths)

# Prepare labels for training
y_train_ages, y_train_genders = extract_labels(train_image_paths, all_image_paths, ages, genders)

# Prepare labels for validation
y_val_ages, y_val_genders = extract_labels(val_image_paths, all_image_paths, ages, genders)

# Prepare labels for test dataset
y_test_ages, y_test_genders = extract_labels(test_image_paths, all_image_paths, ages, genders)

# Check the shapes to confirm
print(f'y_train_ages shape: {y_train_ages.shape}')
print(f'y_train_genders shape: {y_train_genders.shape}')
print(f'y_val_ages shape: {y_val_ages.shape}')
print(f'y_val_genders shape: {y_val_genders.shape}')
print(f'y_test_ages shape: {y_test_ages.shape}')
print(f'y_test_genders shape: {y_test_genders.shape}')
